## Initialize environment

Note: Before executing this notebook on colab, copy your .env file in /content/drive/MyDrive/.env or otherwise you'll be prompted to enter your GEMINI_API_KEY all the times.

In [5]:
! uv pip install agentics-py

import os
import sys
from getpass import getpass

from dotenv import find_dotenv, load_dotenv

CURRENT_PATH = ""

IN_COLAB = "google.colab" in sys.modules
print("In Colab:", IN_COLAB)


if IN_COLAB:
    CURRENT_PATH = "/content/drive/MyDrive/"
    # Mount your google drive
    load_dotenv("/content/drive/MyDrive/.env")
    from google.colab import drive

    drive.mount("/content/drive")
else:
    load_dotenv(find_dotenv())

if not os.getenv("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = getpass("Enter your GEMINI_API_KEY:")

Using Python 3.12.11 environment at: /Users/gliozzo/miniforge3/envs/agentics
Audited 1 package in 15ms
In Colab: False


## Create a new tool
Wrap the Duck Duck Go Search API call into a CrewAI tool using the @tool decorator. Similarly you can turn any function of your choice into a tool. 

In [6]:
from ddgs import DDGS
from crewai.tools import tool
## Define a Crew AI tool to get news for a given date using the DDGS search engine
@tool("fetch_data_async")
def web_search(query: str) -> str:
    """ Fetch web search results for the given query using DDGS."""
    return str(DDGS().text(query, max_results=10))    


## Define your agent, task and crew

Note that the agent is provided with memory = True This provides the agent with conversation history. By activating memory=True in the crew, we make sure that the agents persists across multiple crew session calls, with different input states. 

In [ ]:
from crewai import Agent, Task, Crew, Process

# Create a conversational agent with a friendly role/goal
chat_agent = Agent(
    role="Helpful Assistant",
    goal="Have a natural multi-turn conversation",
    backstory="You are a friendly assistant that remembers context and asks for clarification when needed.",
    memory=True  # enable memory for conversational context. 
)

# Define a simple Task that represents a single AI response
task = Task(
    description="Respond appropriately to user's message, maintaining context. {input}",
    expected_output="A conversational reply",
    agent=chat_agent,
    tools=[web_search]
)

crew = Crew(
    agents=[chat_agent],
    tasks=[task],
    memory=True,  # ensure conversation state persists across turns
)

## Run the crew in a conversation loop
Since memory has been set to True, the crew will get access to the previous round of conversations, exibiting a conversational behaviour

In [ ]:
while user_input := input("You: ").strip():
    print("\n\n===========\nYou: ",user_input)
    result = crew.kickoff(inputs={"input": user_input})
    print("\nAssistant:", result)



You:  Tell me about lisa

Assistant: I need more context to provide a detailed response about "lisa." Are you referring to someone specific, like a public figure, a fictional character, or perhaps something related to a certain topic? Please clarify!


You:  I mean amini

Assistant: "There are quite a few notable references to 'Amini.' One of the most significant is Mahsa Amini, a young Kurdish-Iranian woman whose death in 2022 while in police custody sparked widespread protests in Iran, highlighting issues of women's rights and government oppression. There's also Michael Amini, founder of the Amini Innovation Corp, a luxury home furnishings company. Additionally, there are various meanings associated with the name 'Amini' itself, often denoting concepts like trustworthiness in different cultures. 

What specific aspect of 'amini' were you thinking about? Let me know so I can provide more focused information!"


You:  I'm talking about lisa

Assistant: It looks like you mentioned "li